<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle

    Attributes:
        - state (list of lists): The current state of the puzzle.
        - parent (PuzzleNode): The parent node in the search tree.
        - g (int): The cost from the start node to this node.
        - h (int): The heuristic estimate of the cost from this node to the goal.
        - f (int): The total estimated cost (f = g + h).
    """

    def __init__(self, state, parent=None, g=0, heuristic=None):
        """
        Initialize a PuzzleNode.

        Input:
            -state: The current state of the puzzle (list of lists).
            -parent: The parent node (PuzzleNode).
            -g: The cost from the start node to this node.
            -heuristic: A function that calculates the heuristic value.
        """
        self.state = state
        self.parent = parent  # Store parent node
        self.g = g  # Cost from the start node to this node (g value)
        self.heuristic = heuristic
        self.h = self.heuristic(self.state)  # Heuristic value (h value)
        self.f = self.g + self.h  # Total cost (f = g + h)

    def __str__(self):
        """
        Print the state of the node

        returns:
            -str: node's state
        """
        grid = ""
        n = len(self.state)
        for row in self.state:
            grid += (
                "| "
                + " | ".join(f"{cell:2}" if cell != 0 else "  " for cell in row)
                + " |\n"
            )
        return grid

    def __lt__(self, other):
        """
        Define less-than comparison between two PuzzleNode instances based on their f value.

        returns:
            -bool: True if the current node has a lower f value than the other node, False otherwise.
        """
        return self.f < other.f

    def expand(self):
        """
        Expand the current node.

        returns:
            -children: A list of the child nodes.
        """
        children = []

        # Store the position of the empty tile
        for i in range(len(self.state)):
            for j in range(len(self.state[i])):
                if self.state[i][j] == 0:
                    empty_tile = (i, j)
                    break

        # Generate child nodes by moving nearby tiles to the empty tile
        for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            child_i, child_j = empty_tile[0] + i, empty_tile[1] + j
            # Check if the child node is within the grid
            if 0 <= child_i < len(self.state) and 0 <= child_j < len(self.state[0]):
                # Create a child state by copying the current state
                child_state = [row[:] for row in self.state]
                # Swap the empty tile and the nearby tile
                (
                    child_state[empty_tile[0]][empty_tile[1]],
                    child_state[child_i][child_j],
                ) = (child_state[child_i][child_j], 0)
                # Append the child node to the list and increment the cost
                children.append(
                    PuzzleNode(child_state, self, self.g + 1, self.heuristic)
                )

        return children

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
from queue import Queue


# Create a goal state based on the size of the puzzle
def create_goal_state(n):
    """
    This function creates a goal state based on the size of the puzzle
    Input:
        -n: the size of the puzzle
    Output:
        -goal_state: the goal state (list of lists)
    """
    # Initialize an n x n grid with zeros
    goal_state = [[0 for _ in range(n)] for _ in range(n)]

    # Fill in the grid with goal state values based on the size of the puzzle
    for i in range(n):
        for j in range(n):
            goal_state[i][j] = i * n + j
    return goal_state


# Get the goal positions of tiles
def goal_state_position(n):
    """
    This function returns the position of each tile in the goal state
    Input:
        -n: the size of the puzzle
    Output:
        -position: a dictionary with the position of each tile in the goal state (row, column)
    """
    # Create a goal state based on the size of the puzzle
    goal_state = create_goal_state(n)

    # Initialize a dictionary to store the position of each tile in the goal state
    position = {}

    # Fill in the dictionary with the position of each tile in the goal state
    for i in range(n):
        for j in range(n):
            position[goal_state[i][j]] = (i, j)

    return position


# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    # Create a goal state based on the size of the puzzle
    goal_state = create_goal_state(len(state))

    # Initialize a counter to keep track of the number of misplaced tiles
    counter = 0

    # Count the number of misplaced tiles, comparing the current state with the goal state
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != goal_state[i][j] and state[i][j] != 0:
                counter += 1

    return counter


# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # Create a goal state based on the size of the puzzle
    goal_state = create_goal_state(len(state))

    # Get the position of each tile in the goal state
    goal_position = goal_state_position(len(state))

    # Initialize a counter to keep track of the Manhattan distance
    counter = 0

    # Calculate and get the sum Manhattan distance, comparing the current state with the goal state
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != 0:
                # Calculate the Manhattan distance
                counter += abs(i - goal_position[state[i][j]][0]) + abs(
                    j - goal_position[state[i][j]][1]
                )

    return counter


# EXTENSION 2: Pattern database heuristic
# I have implemented this heuristic specifically for 8-puzzle

from collections import defaultdict
import queue


# Helper function to convert the state to a tuple to store it in the explored set
def state_to_tuple(state):
    return tuple(tuple(row) for row in state)


def get_pattern_database():
    """
    Generates a pattern database for the 8-puzzle by performing a reverse breadth-first search
    from the goal configuration, calculating the minimum moves required to reach each possible state.

    Returns:
        pattern_db (dict): A dictionary mapping each state tuple to its minimum distance from the goal.
    """
    # Initialize pattern database and set a default value of infinity for unexplored states
    pattern_db = defaultdict(dict)

    # Define the goal state as a 3x3 tuple
    goal_state = state_to_tuple(create_goal_state(3))
    # Set distance to 0 for the goal state
    pattern_db[goal_state] = 0

    # Set up queue and initialize with goal state with distance of 0
    q = queue.Queue()
    q.put((goal_state, 0))

    # Perform reverse BFS to calculate minimum moves required to reach each state
    while not q.empty():
        current_state, dist = q.get()

        # Locate the blank position
        for i in range(3):
            for j in range(3):
                if current_state[i][j] == 0:
                    blank_pos = (i, j)
                    break

        # Attempt all possible moves
        for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new_i, new_j = blank_pos[0] + i, blank_pos[1] + j

            # If the new position is within the grid
            if 0 <= new_i < 3 and 0 <= new_j < 3:
                # Swap 0 with adjacent tile to create a new state
                new_state = [list(row) for row in current_state]
                new_state[blank_pos[0]][blank_pos[1]], new_state[new_i][new_j] = (
                    new_state[new_i][new_j],
                    0,
                )
                new_state_tuple = tuple(tuple(row) for row in new_state)

                # Only update if the state hasn't been seen or a shorter distance is found
                if (
                    new_state_tuple not in pattern_db
                    or dist + 1 < pattern_db[new_state_tuple]
                ):
                    pattern_db[new_state_tuple] = dist + 1
                    q.put((new_state_tuple, dist + 1))

    return pattern_db


# Global pattern database for 8-puzzle
PATTERN_DB = get_pattern_database()


# Pattern database heuristic
def h3(state):
    """
    Pattern database heuristic for the 8-puzzle.

    Input:
        -state: Current puzzle state as list of lists
    Output:
        -h: Exact minimum number of moves required to reach goal based on pattern database
    """
    # Convert state to tuple format for database lookup
    state_tuple = state_to_tuple(state)
    return PATTERN_DB[state_tuple]


# Update the heuristics list to include the new heuristic
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

### High-Level Explanation of the Puzzle Solving Algorithm


The function first checks if the provided puzzle state is valid by ensuring it is a square grid and contains the correct set of numbers (if not, it returns error code -1). A priority queue (frontier) is used to manage puzzle states, prioritizing states with the lowest estimated cost (f = g + h), where g is the path cost, and h is the heuristic estimate of remaining moves. The explored states are stored in a set for fast lookup to avoid revisiting states, implementing a graph search that ensures each state is expanded only once. The search continues by expanding nodes with the lowest estimated cost, generating child states, and adding unexplored ones to the frontier until the goal state is reached. If the goal state is found, the function reconstructs the path from the start state to the goal state. The algorithm also tracks the number of nodes expanded and the maximum size of the frontier to provide insight into the search process. If no solution is found, it returns an error code -2 indicating that the puzzle is unsolvable.

### Code Implementation

In [3]:
# Import any packages or define any helper functions you need here
from queue import PriorityQueue


# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """

    # Initialize the number of steps, nodes expanded, maximum size, and the optimal path of the frontier
    steps, exp, max_frontier = 0, 0, 0
    opt_path = []

    # Edge cases: Check whether the state correctly defined, returning error code -1 if not
    # Check whether n > 2, returning error code -1 if not
    if len(state) < 3:
        return steps, exp, max_frontier, None, -1

    # Check whether the gird is n x n, returning error code -1 if not
    for i in range(len(state)):
        if len(state[i]) != len(state):
            return steps, exp, max_frontier, None, -1
    # Check whether it contains every number from 0 to n^2 − 1, returning error code -1 if not
    elements = set(range(len(state) ** 2))
    for row in state:
        elements -= set(row)
    if elements:
        return steps, exp, max_frontier, None, -1

    # Check for solvability (Extension 1)
    # Reference: geeksforgeeks.org/check-instance-8-puzzle-solvable
    # Count the number of inversions
    inversions = 0
    list_of_elements = [
        state[i][j]
        for i in range(len(state))
        for j in range(len(state))
        if state[i][j] != 0
    ]
    for i in range(len(list_of_elements)):
        for j in range(i + 1, len(list_of_elements)):
            if list_of_elements[i] > list_of_elements[j]:
                inversions += 1
    # Check whether N is odd or even
    # If N is odd, the puzzle instance is solvable if the number of inversions is even
    if len(state) % 2 == 1:
        if inversions % 2 == 1:
            return steps, exp, max_frontier, opt_path, -2
    # If N is even, the puzzle is solvable if:
    # The sum of the number of inversions and the row number of the blank tile (counting from the bottom, starting from 1) is even.
    else:
        for i in range(len(state)):
            for j in range(len(state)):
                if state[i][j] == 0:
                    blank_row = len(state) - i
        # Corrected solvability condition
        if (blank_row + inversions) % 2 != 0:
            return steps, exp, max_frontier, opt_path, -2

    # frontier should be a priority queue because we want to pop the node with the lowest f value
    # explored should be a set because we want to check if a node has been explored in O(1) time
    # Initialize the frontier (as priority queue) and explored set
    frontier = PriorityQueue()
    explored = set()

    # Initialize the start node
    start_node = PuzzleNode(state, None, 0, heuristic)
    frontier_size = 1

    # Update the max frontier size
    max_frontier = max(max_frontier, frontier_size)

    # Create a goal state based on the puzzle size
    goal_state = create_goal_state(len(state))

    # Add the start node to the frontier
    frontier.put((start_node.f, start_node))

    # A* Search Algorithm
    # Iterate through the frontier until it is empty
    while not frontier.empty():
        _, current_node = frontier.get()
        current_state = current_node.state

        # Check if the current state is the goal state
        if current_state == goal_state:
            # Reconstruct the optimal path
            while current_node:
                opt_path.append(current_node.state)
                current_node = current_node.parent
            opt_path.reverse()
            # Update the number of steps
            steps = len(opt_path) - 1

            return steps, exp, max_frontier, opt_path, 0

        # If the current state is not the goal state, expand the node and add to the queue
        children = current_node.expand()
        for child in children:
            child_state_tuple = state_to_tuple(child.state)
            if child_state_tuple not in explored:
                frontier.put((child.f, child))
                # Increment the frontier size and update the max frontier size
                frontier_size += 1
                max_frontier = max(max_frontier, frontier_size)

        # Increment the number of nodes expanded
        exp += 1

        # Add the current state to the explored set
        explored.add(state_to_tuple(current_node.state))

    # EXTENSION 1: If the frontier is empty and goal not found, it means the puzzle is unsolveable, return error code -2
    return steps, exp, max_frontier, opt_path, -2

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [4]:
## Test for state not correctly defined

incorrect_state = [[0, 1, 2], [2, 3, 4], [5, 6, 7]]
_, _, _, _, err = solvePuzzle(incorrect_state, lambda state: 0)
assert err == -1

In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = (
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[7, 0, 8], [4, 6, 1], [5, 3, 2]],
    [[5, 7, 6], [2, 4, 3], [8, 1, 0]],
)

# Test the values returned by the heuristic functions
h_mt_vals = [7, 8, 7]
h_man_vals = [15, 17, 18]

for i in range(0, 3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert h_mt == h_mt_vals[i]
    assert h_man == h_man_vals[i]

In [6]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[2, 3, 7], [1, 8, 4], [6, 5, 0]],
    [[2, 3, 7], [1, 8, 4], [6, 0, 5]],
    [[2, 3, 7], [1, 0, 4], [6, 8, 5]],
    [[2, 0, 7], [1, 3, 4], [6, 8, 5]],
    [[0, 2, 7], [1, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [0, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 0, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 4, 0], [6, 8, 5]],
    [[1, 2, 0], [3, 4, 7], [6, 8, 5]],
    [[1, 0, 2], [3, 4, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 0, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 0], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 5], [6, 8, 0]],
    [[1, 4, 2], [3, 7, 5], [6, 0, 8]],
    [[1, 4, 2], [3, 0, 5], [6, 7, 8]],
    [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]],
]

astar_steps = [17, 25, 28]
for i in range(0, 3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[0]
    )
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_mt == steps_man == astar_steps[i]
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert expansions_man < expansions_mt
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert opt_path_mt == opt_path_soln

In [7]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [
    [1, 2, 6, 3],
    [0, 9, 5, 7],
    [4, 13, 10, 11],
    [8, 12, 14, 15],
]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[0]
)
steps_man, expansions_man, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[1]
)
# Test whether the number of optimal steps is correct and the same
assert steps_mt == steps_man == 9
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert expansions_mt >= expansions_man

# Extra Tests

I added more test cases to thoroughly examine the funcionality of the code

### Edge cases: Incorrect Formats

In [8]:
# Test 1: Non-square puzzle
state = [[1, 2, 3], [4, 5, 6]]
steps, exp, max_frontier, opt_path, err = solvePuzzle(state, h1)
# Expected err == -1 since it is not correctly formatted
assert err == -1
assert opt_path is None

# Test 2: Inconsistent row lengths
state = [[1, 2, 3], [4, 5], [6, 7, 8]]
# Expected err == -1 since it is not correctly formatted
assert err == -1
assert opt_path is None

# Test 3: Missing numbers
state = [[1, 2, 3], [4, 5, 6], [7, 0, 0]]  # Missing '8', duplicate '0'
steps, exp, max_frontier, opt_path, err = solvePuzzle(state, h1)
# Expected err == -1 since it is not correctly formatted
assert err == -1

# Test 4: Duplicate numbers
state = [[1, 2, 3], [4, 5, 5], [6, 7, 0]]
steps, exp, max_frontier, opt_path, err = solvePuzzle(state, h1)
# Expected err == -1 since it is not correctly formatted
assert err == -1

# Test 5: Numbers out of range
state = [[1, 2, 3], [4, 5, 9], [6, 7, 0]]
steps, exp, max_frontier, opt_path, err = solvePuzzle(state, h1)
# Expected err == -1 since it is not correctly formatted
assert err == -1

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [9]:
## Puzzle solvability test (3 x 3 grid)

unsolvable_initial_state_3x3 = [[7, 5, 6], [2, 4, 3], [8, 1, 0]]
_, _, _, _, err = solvePuzzle(unsolvable_initial_state_3x3, lambda state: 0)
assert err == -2

In [10]:
## Puzzle solvability test (4 x 4 grid)

unsolvable_initial_state_4x4 = [
    [0, 2, 1, 3],
    [4, 5, 6, 7],
    [8, 9, 10, 11],
    [12, 13, 14, 15],
]
_, _, _, _, err = solvePuzzle(unsolvable_initial_state_4x4, lambda state: 0)
assert err == -2

## Extra heuristic explanation

I used a precomputed database to optimize move estimation. I generated this database by performing a reverse breadth-first search, starting from the goal configuration and calculating the minimum moves required to reach each possible state.

In the breadth-first search process, I explored possible moves of the blank tile, generating new states and recording each one with the fewest moves needed if it is previously unexplored or if it discovers a shorter path. Then, in the heuristic function, I simply look up the stored move count for any current state, allowing for quick, effective estimates.

This heuristic dominates both the number of misplaced tiles and Manhattan distance heuristics because it provides exact minimum moves based on precomputed knowledge, rather than an estimate. While the number of misplaced tiles and Manhattan distance heuristics give approximations based on tile position without accounting for all path constraints, the pattern database leverages actual move counts, thus reducing unnecessary node expansions.

In [11]:
## Extra heuristic function test.
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0, 3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[2]
    )
    steps_man, expansions_man, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_new == steps_man == astar_steps[i]

    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert dom > 0

## Modified Code for Memoization

Since memoization requries multiple modificaitons in the code, I added new code cell for memoization

In [12]:
# Import any packages or define any helper functions you need here
from queue import PriorityQueue

global_heuristic_memo = {}


def retrieve_memoization(state, heuristic):
    """
    This function retrieves the heuristic value for a given state from the global memoization dictionary.

    Inputs:
        - state: The current state of the puzzle (list of lists).
        - heuristic: The heuristic function used to compute the heuristic value.

    Output:
        - h_value: The heuristic value for the given state, either memoized value or calculated value.
    """
    global global_heuristic_memo
    state_tuple = state_to_tuple(state)
    # Check if the state has been memoized
    if state_tuple in global_heuristic_memo:
        # Return the memoized heuristic value
        return global_heuristic_memo[state_tuple]
    # If the state has not been memoized, calculate the heuristic value and memoize it
    else:
        h_value = heuristic(state)
        global_heuristic_memo[state_tuple] = h_value
        return h_value


# Manhattan distance heuristic
# Added h2_counter to keep track of the number of times the heuristic function is called
def h2(state):
    global h2_counter
    h2_counter += 1
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # Create a goal state based on the size of the puzzle
    goal_state = create_goal_state(len(state))

    # Get the position of each tile in the goal state
    goal_position = goal_state_position(len(state))

    # Initialize a counter to keep track of the Manhattan distance
    counter = 0

    # Calculate and get the sum Manhattan distance, comparing the current state with the goal state
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != 0:
                # Calculate the Manhattan distance
                counter += abs(i - goal_position[state[i][j]][0]) + abs(
                    j - goal_position[state[i][j]][1]
                )

    return counter


class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle

    Attributes:
        - state (list of lists): The current state of the puzzle.
        - parent (PuzzleNode): The parent node in the search tree.
        - g (int): The cost from the start node to this node.
        - h (int): The heuristic estimate of the cost from this node to the goal.
        - f (int): The total estimated cost (f = g + h).
    """

    # Modified initialization to require h value in order to store the memoized heuristic value
    def __init__(self, state, parent=None, g=0, h=0):
        """
        Initialize a PuzzleNode.

        Input:
            - state: The current state of the puzzle (list of lists).
            - parent: The parent node (PuzzleNode).
            - g: The cost from the start node to this node.
            - h: The heuristic value.
        """
        self.state = state
        self.parent = parent  # Store parent node
        self.g = g  # Cost from the start node to this node (g value)
        self.h = h  # Heuristic value (h value)
        self.f = self.g + self.h  # Total cost (f = g + h)

    def __str__(self):
        """
        Print the state of the node

        returns:
            - str: node's state
        """
        return str(self.state)

    def __lt__(self, other):
        """
        Define less-than comparison between two PuzzleNode instances based on their f value.

        returns:
            - bool: True if the current node has a lower f value than the other node, False otherwise.
        """
        return self.f < other.f

    def expand(self, heuristic):
        """
        Expand the current node.

        Inputs:
            - heuristic: The heuristic function to compute h values.

        returns:
            - children: A list of the child nodes.
        """
        children = []

        # Store the position of the empty tile
        empty_tile = None
        for i in range(len(self.state)):
            for j in range(len(self.state[i])):
                if self.state[i][j] == 0:
                    empty_tile = (i, j)
                    break
            if empty_tile:
                break  # Exit the outer loop if empty tile is found

        # Generate child nodes by moving nearby tiles to the empty tile
        for i_offset, j_offset in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            child_i, child_j = empty_tile[0] + i_offset, empty_tile[1] + j_offset
            # Check if the child node is within the grid
            if 0 <= child_i < len(self.state) and 0 <= child_j < len(self.state[0]):
                # Create a child state by copying the current state
                child_state = [row[:] for row in self.state]
                # Swap the empty tile and the nearby tile
                (
                    child_state[empty_tile[0]][empty_tile[1]],
                    child_state[child_i][child_j],
                ) = (child_state[child_i][child_j], 0)
                # Compute heuristic value for child_state with memoization function
                h = retrieve_memoization(child_state, heuristic)
                # Append the child node to the list and increment the cost
                children.append(PuzzleNode(child_state, self, self.g + 1, h))

        return children


def solvePuzzle_memo(state, heuristic):
    """
    Modified version of solvePuzzle that uses global memoization to speed up subsequent runs.
    """
    steps, exp, max_frontier = 0, 0, 0
    opt_path = []

    if len(state) < 3:
        return steps, exp, max_frontier, None, -1

    for i in range(len(state)):
        if len(state[i]) != len(state):
            return steps, exp, max_frontier, None, -1

    elements = set(range(len(state) ** 2))
    for row in state:
        elements -= set(row)
    if elements:
        return steps, exp, max_frontier, None, -1

    inversions = 0
    list_of_elements = [
        state[i][j]
        for i in range(len(state))
        for j in range(len(state))
        if state[i][j] != 0
    ]
    for i in range(len(list_of_elements)):
        for j in range(i + 1, len(list_of_elements)):
            if list_of_elements[i] > list_of_elements[j]:
                inversions += 1

    if len(state) % 2 == 1:
        if inversions % 2 == 1:
            return steps, exp, max_frontier, opt_path, -2
    else:
        blank_row = None
        for i in range(len(state)):
            for j in range(len(state)):
                if state[i][j] == 0:
                    blank_row = len(state) - i
                    break
            if blank_row is not None:
                break
        if (blank_row + inversions) % 2 != 0:
            return steps, exp, max_frontier, opt_path, -2

    frontier = PriorityQueue()
    explored = set()

    # Get initial heuristic value using the memoized function
    h = retrieve_memoization(state, heuristic)

    start_node = PuzzleNode(state, None, 0, h)
    frontier_size = 1
    max_frontier = max(max_frontier, frontier_size)

    goal_state = create_goal_state(len(state))

    frontier.put((start_node.f, start_node))

    while not frontier.empty():
        _, current_node = frontier.get()
        frontier_size -= 1
        current_state_tuple = state_to_tuple(current_node.state)

        if current_state_tuple in explored:
            continue

        explored.add(current_state_tuple)

        if current_node.state == goal_state:
            while current_node:
                opt_path.append(current_node.state)
                current_node = current_node.parent
            opt_path.reverse()
            steps = len(opt_path) - 1
            return steps, exp, max_frontier, opt_path, 0

        exp += 1

        children = current_node.expand(heuristic)

        for child in children:
            child_state_tuple = state_to_tuple(child.state)
            if child_state_tuple not in explored:
                frontier.put((child.f, child))
                frontier_size += 1
                max_frontier = max(max_frontier, frontier_size)

    return steps, exp, max_frontier, opt_path, -2

In [13]:
import time
from tabulate import tabulate

global global_heuristic_memo, h2_counter

test_cases = [
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 5, 6], [7, 0, 8]],
    [[1, 2, 3], [4, 5, 6], [0, 7, 8]],
]

results = {
    "first_run": {"times": [], "heuristic_calls": [], "total_time": 0},
    "second_run": {"times": [], "heuristic_calls": [], "total_time": 0},
}

# Clear memoization and run first set
global_heuristic_memo.clear()
first_run_start = time.time()

for case in test_cases:
    h2_counter = 0
    case_start = time.time()
    steps, exp, max_frontier, path, err = solvePuzzle_memo(case, h2)
    case_time = time.time() - case_start
    results["first_run"]["times"].append(case_time)
    results["first_run"]["heuristic_calls"].append(h2_counter)

results["first_run"]["total_time"] = time.time() - first_run_start

# Run second set with memoization
second_run_start = time.time()

for case in test_cases:
    h2_counter = 0
    case_start = time.time()
    steps, exp, max_frontier, path, err = solvePuzzle_memo(case, h2)
    case_time = time.time() - case_start
    results["second_run"]["times"].append(case_time)
    results["second_run"]["heuristic_calls"].append(h2_counter)

results["second_run"]["total_time"] = time.time() - second_run_start

# Create detailed comparison table
detailed_headers = [
    "Case",
    "First Run Time",
    "Second Run Time",
    "Time Improvement",
    "First Run Heuristic Calls",
    "Second Run Heuristic Calls",
    "Heuristic Calls Reduction",
]
detailed_data = []

for i in range(len(test_cases)):
    first_time = results["first_run"]["times"][i]
    second_time = results["second_run"]["times"][i]
    first_calls = results["first_run"]["heuristic_calls"][i]
    second_calls = results["second_run"]["heuristic_calls"][i]

    time_reduction = (
        ((first_time - second_time) / first_time * 100) if first_time > 0 else 0
    )
    calls_reduction = (
        ((first_calls - second_calls) / first_calls * 100) if first_calls > 0 else 0
    )

    detailed_data.append(
        [
            f"Case {i+1}",
            f"{first_time:.4f}s",
            f"{second_time:.4f}s",
            f"{time_reduction:.1f}%",
            first_calls,
            second_calls,
            f"{calls_reduction:.1f}%",
        ]
    )

print("\nPerformance Comparison:")
print(tabulate(detailed_data, headers=detailed_headers, tablefmt="grid"))


Performance Comparison:
+--------+------------------+-------------------+--------------------+-----------------------------+------------------------------+-----------------------------+
| Case   | First Run Time   | Second Run Time   | Time Improvement   |   First Run Heuristic Calls |   Second Run Heuristic Calls | Heuristic Calls Reduction   |
+========+==================+===================+====================+=============================+==============================+=============================+
| Case 1 | 0.0251s          | 0.0070s           | 72.3%              |                         861 |                            0 | 100.0%                      |
+--------+------------------+-------------------+--------------------+-----------------------------+------------------------------+-----------------------------+
| Case 2 | 0.0167s          | 0.0116s           | 30.8%              |                         603 |                            0 | 100.0%                      |
+--

# References

- GeeksforGeeks. (2022, July 26). "How to check if an instance of 8 puzzle is solvable?" Retrieved from https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable/
    - Used for checking solvability (error -2)